In [2]:
import torch.optim as optim
import torch
import torch.nn as nn
import torch.nn.parallel
import torch.utils.data
import torch.utils.data.distributed
import torchvision.transforms as transforms
import import_ipynb
from torchvision import datasets
from torch.autograd import Variable
from model.resnet import ResNet18
from model.resnet_l import ResNet50

import json
import os

importing Jupyter notebook from /home/u2022171199/jupyterlab/KDtest/model/resnet_l.ipynb


In [3]:
import Ranger
from tqdm import tqdm
import numpy as np
from matplotlib import pyplot as plt
import timm
from torchinfo import summary #打印网络模型各层信息的库

In [4]:
def get_dataloader(train_dir, val_dir, batch_size):
    transform = transforms.Compose([
        transforms.RandomRotation(10),
        transforms.GaussianBlur(kernel_size=(5, 5), sigma=(0.1, 3.0)),
        transforms.ColorJitter(brightness=0.5, contrast=0.5, saturation=0.5),
        transforms.Resize((224, 224)),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.44127703, 0.4712498, 0.43714803], std=[0.18507297, 0.18050247, 0.16784933])
    ])
    
    transform_test = transforms.Compose([
        transforms.Resize((224, 224)),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.44127703, 0.4712498, 0.43714803], std=[0.18507297, 0.18050247, 0.16784933])
    ])

    dataset_train = datasets.ImageFolder(train_dir, transform=transform)
    dataset_test = datasets.ImageFolder(val_dir, transform=transform_test)
    with open('class.txt', 'w', encoding='utf-8') as file:
        file.write(str(dataset_train.class_to_idx))
    with open('class.json', 'w', encoding='utf-8') as file:
        file.write(json.dumps(dataset_train.class_to_idx))
    train_loader = torch.utils.data.DataLoader(dataset_train, batch_size = batch_size, shuffle=True)
    test_loader = torch.utils.data.DataLoader(dataset_test, batch_size = batch_size, shuffle=False)
    return train_loader,test_loader

In [25]:
def train(model, train_dataloader, test_dataloader, 
          device, batch_size, num_epoch, learning_rate, criterion,  optim, init=False, scheduler_type='Cosine'):
    def init_xavier(m):
        if type(m) == nn.Linear:
            nn.init.xavier_normal_(m.weight)
    
    if init:
        net.apply(init_xavier)
    
    print('trainning on:', device)
    model.to(device)
    
    if optim == 'sgd':
        optimizer = torch.optim.SGD(model.parameters(), lr = learning_rate)
    if optim == 'adam':
        optimizer = torch.optim.Adam(model.parameters(), lr = learning_rate)
    if optim == 'adamW':
        optimizer = torch.optim.AdamW(model.parameters(), lr = learning_rate)
    else:
        optimizer = torch.optim.Adam(model.parameters(), lr = learning_rate)
    if scheduler_type == 'Cosine':
        scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer=optimizer, T_max=20,eta_min=1e-9)
        
    train_losses = []
    train_acces = []
    eval_acces = []
    best_acc = 0.0
    
    for epoch in range(1, num_epoch+1):
        print("——————第 {} 轮训练开始——————".format(epoch))
        model.train()
        total_correct = 0
        total_loss = 0
        for  (data, target) in tqdm(train_dataloader, desc = 'trainning'):
            data, target = data.to(device), target.to(device)
            _, _, _, _, _, out = model(data)
            loss = criterion(out, target)
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
            
            total_loss += loss.data.item()
            _, pred = torch.max(out.data, 1)
            total_correct += torch.sum(pred==target).item()
        avg_loss = total_loss / len(train_dataloader)
        accuracy = total_correct / len(train_dataloader.dataset)
        print('Epoch: {}, test_loss: {:.4f}, train_accuracy: {}/{} ({:.0f}%)'.format(epoch,
            avg_loss, total_correct, len(train_dataloader.dataset), 100 * accuracy))
        train_acces.append(accuracy)
        train_losses.append(avg_loss)
        scheduler.step()
        
        #验证以及测试步骤
        model.eval()
        total_correct = 0
        total_loss = 0
        with torch.no_grad():
            for data, target in test_dataloader:
                data, target = data.to(device), target.to(device)
                _, _, _, _, _, out = model(data)
                loss = criterion(out, target)
                _,pred = torch.max(out.data, 1)
                total_correct += torch.sum(pred == target).item()
                total_loss += loss.data.item()
            avg_loss = total_loss / len(test_dataloader)
            accuracy = total_correct / len(test_dataloader.dataset)
            if accuracy > best_acc:
                best_acc = accuracy
                torch.save(model.state_dict(), file_dir + '/' +'_best.pth')
            eval_acces.append(accuracy)
            print('Epoch: {}, test_loss: {:.4f}, test_accuracy: {}/{} ({:.0f}%)\n'.format(epoch,
            avg_loss, total_correct, len(test_dataloader.dataset), 100 * accuracy))
    return train_losses, train_acces, eval_acces

In [16]:
if __name__ == '__main__':
    file_dir = 'studentNet'
    if not os.path.isdir(file_dir):
        os.makedirs(file_dir)
    train_dir = './data/train'
    test_dir = './data/val'
    train_dataloader,test_dataloader = get_dataloader(train_dir,test_dir,batch_size=64)
    

In [20]:
    learning_rate = 1e-4
    batch_size = 64
    epoch = 100
    device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
    optimizer = 'adam'

In [21]:
    model = ResNet18()
    model.fc = nn.Linear(model.fc.in_features,12)
    criterion = nn.CrossEntropyLoss()

In [24]:
    train_losses, train_acces, eval_acces = train(model,train_dataloader,test_dataloader,
          device,batch_size,epoch,learning_rate,criterion,optimizer)
    

trainning on: cuda:0
——————第 1 轮训练开始——————


trainning: 100%|██████████| 62/62 [01:45<00:00,  1.70s/it]


Epoch: 1, test_loss: 0.1913, train_accuracy: 3696/3961 (93%)
Epoch: 1, test_loss: 1.0204, test_accuracy: 1146/1698 (67%)

——————第 2 轮训练开始——————


trainning: 100%|██████████| 62/62 [01:49<00:00,  1.77s/it]


Epoch: 2, test_loss: 0.1718, train_accuracy: 3731/3961 (94%)
Epoch: 2, test_loss: 1.9143, test_accuracy: 994/1698 (59%)

——————第 3 轮训练开始——————


...

Epoch: 98, test_loss: 0.0025, train_accuracy: 3960/3961 (100%)
Epoch: 98, test_loss: 0.2809, test_accuracy: 1588/1698 (94%)

——————第 99 轮训练开始——————


trainning: 100%|██████████| 62/62 [01:42<00:00,  1.66s/it]


Epoch: 99, test_loss: 0.0023, train_accuracy: 3960/3961 (100%)
Epoch: 99, test_loss: 0.2857, test_accuracy: 1588/1698 (94%)

——————第 100 轮训练开始——————


trainning: 100%|██████████| 62/62 [01:45<00:00,  1.70s/it]


Epoch: 100, test_loss: 0.0015, train_accuracy: 3961/3961 (100%)
Epoch: 100, test_loss: 0.2885, test_accuracy: 1586/1698 (93%)

